# Distributed Computing with TensorFlow

https://learningtensorflow.com/lesson11/

```
import tensorflow as tf
cluster = tf.train.ClusterSpec({"local": ["localhost:2222", "localhost:2223"]})
server = tf.train.Server(cluster, job_name="local", task_index=1)
server.start()
server.join()
```

In [1]:
import tensorflow as tf

In [2]:
cluster = tf.train.ClusterSpec({"local": ["localhost:2222", "localhost:2223"]})

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/mnist", one_hot=True)

Extracting /tmp/mnist/train-images-idx3-ubyte.gz
Extracting /tmp/mnist/train-labels-idx1-ubyte.gz
Extracting /tmp/mnist/t10k-images-idx3-ubyte.gz
Extracting /tmp/mnist/t10k-labels-idx1-ubyte.gz


In [4]:
x = tf.placeholder(tf.float32, [None, 784])

In [5]:
with tf.device("/job:local/task:1"):
    W = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros([10]))

In [6]:
with tf.device("/job:local/task:0"):
    y = tf.nn.softmax(tf.matmul(x, W) + b)

In [7]:
y_truth = tf.placeholder(tf.float32, [None, 10])

In [8]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_truth * tf.log(y), reduction_indices=[1]))

In [9]:
train = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [24]:
session = tf.Session("grpc://localhost:2222")
globalv = tf.global_variables_initializer()

session.run(globalv)

In [28]:
for _ in range(2000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  session.run(train, feed_dict={x: batch_xs, y_truth: batch_ys})

In [26]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_truth,1))

In [27]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
value = session.run(accuracy, feed_dict={x: mnist.test.images, y_truth: mnist.test.labels})

print(value)

0.9239
